## [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [1]:
# !pip install openai
# !pip install --upgrade langchain

In [84]:
from imports import *
import openai
import langchain

In [85]:
import os
os.environ["OPENAI_API_KEY"] = API_KEY
openai.api_key = API_KEY

## Helper code


This helper function will make it easier to use prompts and look at the generated outputs. 

In [138]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [140]:
text=get_completion("what is 1+1")
text

'1 + 1 equals 2.'

# Prompting Techniques

## 1: Write clear and specific instruction



### 1.1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be : ```, """, < >, `:`

In [89]:
text = f"""
Mitochondria are cellular organelles crucial for energy production\
through cellular respiration, generating adenosine triphosphate (ATP).\
Found in most eukaryotic cells, they also influence cell cycle regulation,\
apoptosis, and maintain cellular homeostasis. Mitochondria possess their \
own DNA, likely originating from a symbiotic relationship between \
ancestral cells and engulfed bacteria.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

Mitochondria are organelles that produce energy through cellular respiration, influence cell cycle regulation and apoptosis, maintain cellular homeostasis, and have their own DNA likely originating from a symbiotic relationship with bacteria.


### 1.2: Ask for a structured output
- JSON, HTML

In [141]:
prompt = f"""
Generate a list of three food items along \ 
with their origin and required ingredients. 
Provide them in JSON format with the following keys: 
name, origin, ingredients.
"""
response = get_completion(prompt)
print(response)

{
  "foodItems": [
    {
      "name": "Sushi",
      "origin": "Japan",
      "ingredients": "Rice, seaweed, fish or vegetables"
    },
    {
      "name": "Tacos",
      "origin": "Mexico",
      "ingredients": "Tortillas, meat or beans, salsa, cheese, lettuce"
    },
    {
      "name": "Pizza",
      "origin": "Italy",
      "ingredients": "Dough, tomato sauce, cheese, various toppings"
    }
  ]
}


### 1.3: Ask the model to check whether conditions are satisfied

In [91]:
text_1 = f"""
Creating scrambled eggs is a breeze! \
Start by heating a pan. While it warms up, \
crack eggs into a bowl and whisk them.\
Once the pan is hot, add a bit of butter and \
pour in the beaten eggs. Stir continuously until\
they're cooked to your liking. Season with salt and \
pepper. Voila! Tasty scrambled eggs ready to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print(response)

Step 1 - Start by heating a pan.
Step 2 - While it warms up, crack eggs into a bowl and whisk them.
Step 3 - Once the pan is hot, add a bit of butter and pour in the beaten eggs.
Step 4 - Stir continuously until they're cooked to your liking.
Step 5 - Season with salt and pepper.
Step 6 - Voila! Tasty scrambled eggs ready to enjoy.


In [92]:
text_2 = f"""
Today's sunlight is vibrant, accompanied by the \
cheerful melody of birds. It's an ideal day for \
a stroll in the park. Blooms adorn the surroundings, \
and trees sway gently in the breeze. Folks relish \
the pleasant weather—picnicking, playing games, \
or unwinding on the grass. A perfect day to connect with nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


### 1.4: Use examples for reference

In [93]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: ```Arr matey! The river what carves the /
deepest valley flows from a humble spring, savvy? /
The grandest symphony starts with a single note, arr! /
And the most intricate tapestry, aye, it begins with a /
solitary thread. Patience, me heartie, be the key to uncoverin' /
treasures hidden in time's chest, aye, the quiet strength /
that be as steady as the wind in our sails.```

<child>: Teach me about inner peace.
"""
response = get_completion(prompt)
print(response)

<grandparent>: ```Ahoy there, me little one! Inner peace be like /
the calm sea on a clear summer's day, ye see? /
It be findin' tranquility amidst the stormy waves, /
and holdin' onto serenity when the world be in disarray. /
It be lettin' go of worries and findin' contentment /
in the present moment, me heartie. Inner peace be /
the anchor that keeps us steady in the face of adversity, /
and the compass that guides us to a peaceful heart.```


## 2: The model needs more time!!

### 2.1: Specify the steps required to complete a task

In [94]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - Jack and Jill, siblings, go on a quest to fetch water from a hilltop well, but encounter misfortune when Jack trips on a stone and tumbles down the hill, with Jill following suit, yet they return home and remain undeterred in their adventurous spirits.

2 - Jack et Jill, frère et sœur, partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent un malheur lorsque Jack trébuche sur une pierre et dévale la colline, suivi de Jill, pourtant ils rentrent chez eux et restent déterminés dans leur esprit d'aventure.

3 - Jack, Jill

4 - {
  "french_summary": "Jack et Jill, frère et sœur, partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent un malheur lorsque Jack trébuche sur une pierre et dévale la colline, suivi de Jill, pourtant ils rentrent chez eux et restent déterminés dans leur esprit d'aventure.",
  "num_names": 2
}


### 2.2: Instruct the model to work out its own solution before rushing to a conclusion

In [155]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a new house with a swimming pool. I need \
 help working out the financials. 
- Land costs $100 / square foot \
- swimming pool costs 50$ / square foot \
- I negotiated a contract for maintenance that will cost \ 
me a flat $10k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. swimming pool cost: 50x
3. Maintenance cost: 10,000 + 100x
Total cost: 100x + 50x + 10,000 + 100x = 150x + 10,000
"""
response = get_completion(prompt)
print(response)


The student's solution is correct. The total cost for the first year of operations is indeed 150x + 10,000.


In [98]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem \
including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a new house with a swimming pool. I need 
help working out the financials. 
- Land costs $100 / square foot \
- swimming pool costs 50$ / square foot \
- I negotiated a contract for maintenance that will cost me a 
flat $10k per year, and an additional $10 / square foot
What is the total cost for the first year of operations 
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. swimming pool cost: 250x
3. Maintenance cost: 10,000 + 100x
Total cost: 100x + 50x + 10,000 + 100x = 150x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Costs:
1. Land cost: $100 * x
2. Swimming pool cost: $50 * x
3. Maintenance cost: $10,000 + $10 * x
Total cost: $100 * x + $50 * x + $10,000 + $10 * x = $160 * x + $10,000

Is the student's solution the same as actual solution just calculated:
No

Student grade:
Incorrect


### 2.4: Generate a marketing product description from a product fact sheet

In [142]:
fact_sheet_chair = """
OVERVIEW
- Part of a stylish collection of modern lounge furniture, 
featuring sofas, coffee tables, ottomans, and more.
- Choose from a variety of upholstery colors and leg finishes.
- Upholstered in plush fabric (CC-200) or premium leather (CC-210) 
with 15 fabric and 8 leather options.
- Leg finish choices include brushed brass, walnut, 
matte black, or chrome.
- Chair is customizable with or without armrests.
- Ideal for home or office environments.
- Suitable for commercial use.

CONSTRUCTION
- Sturdy 4-leg wooden base.
- Smooth recline mechanism for ultimate comfort.

DIMENSIONS
- WIDTH 60 CM | 23.62”
- DEPTH 55 CM | 21.65”
- HEIGHT 75 CM | 29.53”
- SEAT HEIGHT 40 CM | 15.75”
- SEAT DEPTH 50 CM | 19.69”

OPTIONS
- Swivel or fixed base options.
- Choose between soft or firm seat cushions: 
  standard (2.0 lb/ft3) or plush (2.5 lb/ft3).
- Armless design or opt for sleek metal armrests.

MATERIALS
FRAME UPHOLSTERY
- Solid Beech Wood with a lacquered finish.
- Upholstered in premium-grade velvet.

COUNTRY OF ORIGIN
- Denmark
"""

In [143]:
prompt = f"""
Create a catchy product description for our website \
using the technical specs in the triple backticks. \
Transform the details into engaging content that emphasizes \
the product's benefits and uniqueness. \
Focus on making it appealing to our target audience.


Technical details: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Introducing our stunning and versatile lounge chair, a must-have addition to any modern space. Part of our stylish collection of contemporary lounge furniture, this chair is designed to elevate your home or office environment with its sleek and sophisticated aesthetic.

Customization is key with our lounge chair. Choose from a variety of upholstery colors and leg finishes to perfectly match your existing decor. Upholstered in plush fabric or premium leather, you have the option to select from 15 fabric and 8 leather choices, ensuring that your chair is as unique as you are. The leg finish choices include brushed brass, walnut, matte black, or chrome, allowing you to create a truly personalized look.

Not only is this chair visually stunning, but it is also built to last. The sturdy 4-leg wooden base provides stability and durability, ensuring that your chair will stand the test of time. The smooth recline mechanism guarantees ultimate comfort, allowing you to relax and unwind after a l

#### The text is too long 

In [144]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Introducing our modern lounge chair, part of a stylish collection of furniture for home or office. Customize your chair with a choice of upholstery colors, leg finishes, and armrest options. With a sturdy wooden base and smooth recline mechanism, this chair offers ultimate comfort. Made in Denmark.


In [145]:
len(response.split())

47

#### The text does not have the required details

In [146]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our modern lounge chair, part of a stylish collection of furniture for home or office environments. Choose from a variety of upholstery colors and leg finishes, including plush fabric or premium leather. The sturdy wooden base and smooth recline mechanism ensure ultimate comfort. Made with solid Beech Wood and premium-grade velvet upholstery, this chair is both durable and luxurious. Available with or without armrests, and in swivel or fixed base options. Made in Denmark.


In [106]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 6-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our modern lounge chair, part of a stylish collection of furniture for both home and office environments. This chair offers customizable options, including upholstery choices of plush fabric or premium leather, with a variety of colors available. The sturdy wooden base and smooth recline mechanism ensure ultimate comfort. Choose between swivel or fixed base options, and opt for soft or firm seat cushions. With its sleek design and high-quality materials, this chair is perfect for any space. Product ID: CC-200, CC-210.


#### Ask it to extract information and organize it in a table.

In [147]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
  <h2>Product Description</h2>
  <p>
    Introducing our stylish collection of modern lounge furniture, 
    perfect for furniture retailers looking to offer their customers 
    a contemporary and comfortable seating option. This collection 
    includes sofas, coffee tables, ottomans, and more, allowing 
    customers to create a cohesive and stylish look in their space. 
    With a variety of upholstery colors and leg finishes to choose 
    from, customers can customize their furniture to match their 
    personal style and existing decor.
  </p>
  <p>
    Our chairs are upholstered in either plush fabric (CC-200) or 
    premium leather (CC-210), offering customers 15 fabric options 
    and 8 leather options to choose from. The upholstery materials 
    are of the highest quality, ensuring durability and comfort. 
    Customers can also select their preferred leg finish, with 
    choices including brushed brass, walnut, matte black, or chrome. 
    Additionally, our chairs

In [150]:
from IPython.display import display, HTML

In [152]:
display(HTML(response))

Dimension,Measurement (inches)
Width,"23.62"""
Depth,"21.65"""
Height,"29.53"""
Seat Height,"15.75"""
Seat Depth,"19.69"""


# Summarize

In [111]:
prod_review = """
Grabbed this panda plush toy for my daughter's \
birthday—she's obsessed! This cuddly companion \
goes everywhere with her. It's soft and incredibly \
adorable, with a friendly face. However, it's a tad \
smaller than expected for the price. I reckon there \
might be larger options available for the same cost. \
Surprisingly, it arrived a day early, giving me a \
chance to enjoy some playtime before handing it over.\
"""

## set a word limit

In [112]:
prompt = f"""
Your task is to generate a short \
summary of a product \
review from an ecommerce site. 

Summarize the review below, delimited by
triple backticks, in at most 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)


This panda plush toy is soft, adorable, and arrived early. However, it is smaller than expected for the price.


## Review specific for shipping and delivery

In [157]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
Shipping deparmtment. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that mention shipping and delivery of the product. 

Review: ```{prod_review}```
"""

The customer loved the panda plush toy but found it smaller than expected for the price. However, they were pleasantly surprised by the early delivery.


## Review for the pricing department

In [158]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
pricing deparmtment, responsible for determining the
price of the product.  

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any 
aspects that are relevant to the price and perceived
value. 

Review: ```{prod_review}```
"""

The panda plush toy is soft, adorable, and arrived early. However, it is smaller than expected for the price, suggesting that larger options may be available for the same cost.


## Try using extract instead of Summary when you want specific information


In [115]:
prompt = f"""
Your task is to extract relevant information from \ 
a product review from an ecommerce site to give \
feedback to the Shipping department. 

From the review below, delimited by triple quotes \
extract the information relevant to shipping and \ 
delivery. Limit to 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

The product arrived a day early, allowing for extra playtime. The size was smaller than expected, suggesting larger options may be available for the same price.


## Summarize multiple product reviews

In [116]:

review_1 = prod_review 

# review for a standing lamp
review_2 = """
Needed a nice lamp for my bedroom, and this one \
had additional storage and not too high of a price \
point. Got it fast - arrived in 2 days. The string \
to the lamp broke during the transit and the company \
happily sent over a new one. Came within a few days \
as well. It was easy to put together. Then I had a \
missing part, so I contacted their support and they \
very quickly got me the missing piece! Seems to me \
to be a great company that cares about their customers \
and products. 
"""

# review for an electric toothbrush
review_3 = """
My dental hygienist recommended an electric toothbrush, \
which is why I got this. The battery life seems to be \
pretty impressive so far. After initial charging and \
leaving the charger plugged in for the first week to \
condition the battery, I've unplugged the charger and \
been using it for twice daily brushing for the last \
3 weeks all on the same charge. But the toothbrush head \
is too small. I’ve seen baby toothbrushes bigger than \
this one. I wish the head was bigger with different \
length bristles to get between teeth better because \
this one doesn’t.  Overall if you can get this one \
around the $50 mark, it's a good deal. The manufactuer's \
replacements heads are pretty expensive, but you can \
get generic ones that're more reasonably priced. This \
toothbrush makes me feel like I've been to the dentist \
every day. My teeth feel sparkly clean! 
"""

# review for a blender
review_4 = """
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

reviews = [review_1, review_2, review_3, review_4]



In [117]:
for i in range(len(reviews)):
    prompt = f"""
    Your task is to generate a short summary of a product \ 
    review from an ecommerce site. 

    Summarize the review below, delimited by triple \
    backticks in at most 20 words. 

    Review: ```{reviews[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")


0 A small, adorable panda plush toy that is loved by a child, but could be larger for the price. 

1 Great lamp with storage, fast delivery, excellent customer service, and easy assembly. Highly recommended. 

2 The reviewer recommends the electric toothbrush for its impressive battery life, but criticizes the small brush head. 

3 The reviewer found the price increase after the sale disappointing and noticed a decrease in quality. 



# Inferring
In this lesson, you will infer sentiment and topics from product reviews and news articles.


## Product review

In [118]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## Sentiment (positive/negative)

In [119]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the product review is positive.


In [120]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

positive


## Identify types of emotions

In [121]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)


satisfied, pleased, grateful, impressed, happy


## Identify anger

In [122]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

No


## Extract product and company name from customer reviews

In [123]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "lamp",
  "Brand": "Lumina"
}


## Doing multiple tasks at once

In [124]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp",
  "Brand": "Lumina"
}


## Inferring topics

In [11]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

## Infer 5 topics

In [125]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

1. Government survey
2. Department satisfaction rating
3. NASA
4. Social Security Administration
5. Job satisfaction improvement


In [126]:
response.split(sep=',')

['1. Government survey\n2. Department satisfaction rating\n3. NASA\n4. Social Security Administration\n5. Job satisfaction improvement']

In [127]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

## create alert notification for certain topics

In [128]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

[1, 0, 0, 1, 1]


In [129]:
if eval(response)[0] == 1:
    print("NASA news ALERT!")

NASA news ALERT!


# Transforming



 we will explore how to use Large Language Models for text transformation tasks such as language translation, spelling and grammar checking, tone adjustment, and format conversion.

## Translation

In [131]:
prompt = f"""
Translate the following English text to Spanish: \ 
```Hi, I would like a table for two```
"""
response = get_completion(prompt)
print(response)

Hola, me gustaría una mesa para dos.


In [132]:
prompt = f"""
Tell me which language this is: 
```Hola, me gustaría una mesa para dos.```
"""
response = get_completion(prompt)
print(response)

This language is Spanish.


In [133]:
prompt = f"""
Translate the following  text to French and Spanish
and English pirate slang: \
```The black pearl is the fastest ship in existance, understand?```
"""
response = get_completion(prompt)
print(response)

French: ```Le perle noire est le navire le plus rapide qui existe, tu comprends?```
Spanish: ```La perla negra es el barco más rápido que existe, ¿entiendes?```
English Pirate Slang: ```Arrr, the black pearl be the swiftest ship in all the seven seas, savvy?```


In [134]:
prompt = f"""
Translate the following text to Spanish in both the \
formal and informal forms: 
'Would you like to order a phone?'
"""
response = get_completion(prompt)
print(response)

Formal: ¿Le gustaría ordenar un teléfono?
Informal: ¿Te gustaría ordenar un teléfono?


## Tone Transformation
Writing can vary based on the intended audience. ChatGPT can produce different tones.


In [135]:
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is MJ, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response)

Dear Sir/Madam,

I hope this letter finds you well. My name is MJ, and I am writing to bring your attention to a specification document regarding a standing lamp.

I kindly request that you take a moment to review the attached spec, as it contains important details about the standing lamp in question.

Thank you for your time and consideration. I look forward to hearing from you soon.

Sincerely,
MJ


## Format Conversion
ChatGPT can translate between formats. The prompt should describe the input and output formats.

In [41]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)

<!DOCTYPE html>
<html>
<head>
<style>
table {
  font-family: arial, sans-serif;
  border-collapse: collapse;
  width: 100%;
}

td, th {
  border: 1px solid #dddddd;
  text-align: left;
  padding: 8px;
}

tr:nth-child(even) {
  background-color: #dddddd;
}
</style>
</head>
<body>

<h2>Restaurant Employees</h2>

<table>
  <tr>
    <th>Name</th>
    <th>Email</th>
  </tr>
  <tr>
    <td>Shyam</td>
    <td>shyamjaiswal@gmail.com</td>
  </tr>
  <tr>
    <td>Bob</td>
    <td>bob32@gmail.com</td>
  </tr>
  <tr>
    <td>Jai</td>
    <td>jai87@gmail.com</td>
  </tr>
</table>

</body>
</html>


In [136]:
from IPython.display import display, Markdown, Latex, HTML, JSON


In [137]:
display(HTML(response))

## Spellcheck/Grammar check.

Here are some examples of common grammar and spelling problems and the LLM's response. 

To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.

In [43]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

The girl with the black and white puppies has a ball.
No errors found.
No errors found.
There goes my freedom. They're going to bring their suitcases.
You're going to need your notebook.
That medicine affects my ability to sleep. Have you heard of the butterfly effect?
This phrase is to check chatGPT for spelling ability.


In [45]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

Got this for my daughter for her birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. However, one of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. Additionally, it's a bit small for what I paid for it. I believe there might be other options that are bigger for the same price. On the positive side, it arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.


In [ ]:
# !pip install redlines

In [47]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

Got this for my daughter for her birthday <span style='color:red;font-weight:700;text-decoration:line-through;'>cuz </span><span style='color:green;font-weight:700;'>because </span>she keeps taking mine from my <span style='color:red;font-weight:700;text-decoration:line-through;'>room.  </span><span style='color:green;font-weight:700;'>room. </span>Yes, adults also like pandas <span style='color:red;font-weight:700;text-decoration:line-through;'>too.  </span><span style='color:green;font-weight:700;'>too. </span>She takes it everywhere with her, and it's super soft and <span style='color:red;font-weight:700;text-decoration:line-through;'>cute.  One </span><span style='color:green;font-weight:700;'>cute. However, one </span>of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. <span style='color:red;font-weight:700;text-decoration:line-through;'>It's </span><span style='color:green;font-weight:700;'>Additionally, it's </span>a bit small for what I paid for <span style='color:red;font-weight:700;text-decoration:line-through;'>it though. </span><span style='color:green;font-weight:700;'>it. </span>I <span style='color:red;font-weight:700;text-decoration:line-through;'>think </span><span style='color:green;font-weight:700;'>believe </span>there might be other options that are bigger for the same <span style='color:red;font-weight:700;text-decoration:line-through;'>price.  It </span><span style='color:green;font-weight:700;'>price. On the positive side, it </span>arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.

In [48]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

**Title: A Delightful Gift for Panda Enthusiasts: A Review of the Soft and Adorable Panda Plush Toy**

*Reviewer: [Your Name]*

---

I recently purchased this charming panda plush toy as a birthday gift for my daughter, who has a penchant for "borrowing" my belongings from time to time. As an adult, I must admit that I too have fallen under the spell of these lovable creatures. This review aims to provide an in-depth analysis of the product, catering to advanced readers who appreciate a comprehensive evaluation.

First and foremost, the softness and cuteness of this panda plush toy are simply unparalleled. Its irresistibly plush exterior makes it a joy to touch and hold, ensuring a delightful sensory experience for both children and adults alike. The attention to detail is evident, with its endearing features capturing the essence of a real panda. However, it is worth noting that one of the ears appears to be slightly asymmetrical, which may not have been an intentional design choice.

While the overall quality of the product is commendable, I must express my slight disappointment regarding its size in relation to its price. Considering the investment made, I expected a larger plush toy. It is worth exploring alternative options that offer a more substantial size without compromising on the price point. Nevertheless, the undeniable charm and softness of this panda plush toy compensate for its relatively smaller dimensions.

In terms of delivery, I was pleasantly surprised to receive the product a day earlier than anticipated. This unexpected early arrival allowed me to indulge in some personal playtime with the panda plush toy before presenting it to my daughter. This added bonus further enhanced my overall satisfaction with the purchase.

In conclusion, this panda plush toy is a delightful gift for both children and adults who appreciate the enchanting allure of these beloved creatures. Its softness, cuteness, and attention to detail make it a truly captivating addition to any collection. While the size may not fully justify the price, the overall quality and timely delivery contribute to a positive purchasing experience. I highly recommend this panda plush toy to anyone seeking a charming and huggable companion.

---

**References**

*Please note that APA style does not require references for personal reviews.

# Opportunities

## Customize the automated reply to a customer email

In [50]:
sentiment = "negative"

# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal sale for around $49 in the month of November, about 
half off, but for some reason (call it price gouging) around the second week of December the prices all went 
up to about anywhere from between $70-$89 for the same system. And the 11 piece system went up around $10 or 
so in price also from the earlier sale price of $29. So it looks okay, but if you look at the base, the part 
where the blade locks into place doesn't look as good as in previous editions from a few years ago, but I 
plan to be very gentle with it (example, I crush very hard items like beans, ice, rice, etc. in the 
blender first then pulverize them in the serving size I want in the blender then switch to the whipping 
blade for a finer flour, and use the cross cutting blade first when making smoothies, then use the flat blade 
if I need them finer/less pulpy). Special tip when making smoothies, finely cut and freeze the fruits and 
vegetables (if using spinach-lightly stew soften the spinach then freeze until ready for use-and if making 
sorbet, use a small to medium sized food processor) that you plan to use that way you can avoid adding so 
much ice if at all-when making your smoothie. After about a year, the motor was making a funny noise. 
I called customer service but the warranty expired already, so I had to buy another one. FYI: The overall 
quality has gone done in these types of products, so they are kind of counting on brand recognition and 
consumer loyalty to maintain sales. Got it in about two days.
"""

In [51]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt)
print(response)

Dear Valued Customer,

Thank you for taking the time to share your review with us. We appreciate your feedback and apologize for any inconvenience you may have experienced.

We are sorry to hear about the price increase you noticed in December. We strive to provide competitive pricing for our products, and we understand your frustration. If you have any further concerns regarding pricing or any other issues, we encourage you to reach out to our customer service team. They will be more than happy to assist you and address any questions or concerns you may have.

We also appreciate your feedback regarding the base of the system. We continuously work to improve the quality of our products, and your comments will be taken into consideration for future enhancements.

Thank you for sharing your special tips and tricks for using our blender. We value your loyalty and are glad to hear that you received your order promptly.

If there is anything else we can assist you with, please do not hesita

## Tell the model to use details from the customer's email

In [54]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}"
"""
response = get_completion(prompt)
print(response)

Dear Valued Customer,

Thank you for taking the time to share your review with us. We appreciate your feedback and apologize for any inconvenience you may have experienced.

We are sorry to hear about the price increase you noticed in December. Our pricing is subject to change based on various factors, including market conditions and demand. However, we understand your concern and will take it into consideration for future pricing adjustments.

Regarding the base of the system, we apologize if it does not meet your expectations. We strive to provide high-quality products, and we appreciate your feedback on this matter. We will pass your comments along to our product development team for further review.

We understand that you have been using the blender for various purposes, and we appreciate your tips for making smoothies. We are sorry to hear that you encountered an issue with the motor after a year of use. As the warranty had expired, we recommend reaching out to our customer servic

# ChatBot

In [57]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [58]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [59]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

Why, to avoid an encounter with the scornful swine on the other side, of course!


In [62]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Jaanav'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Jaanav! It's nice to meet you. How can I assist you today?


In [61]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but as a friendly chatbot, I don't have access to personal information about you, including your name.


In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Jaanav'},
{'role':'assistant', 'content': "Hi Jaanav! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)